In [10]:
from langchain_google_vertexai import ChatVertexAI
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate
)

import os
import json
import random

from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser

from typing import List

## T-PRO

In [2]:
from langchain_openai import ChatOpenAI

T_PRO_CREDS = "../secrets/t-pro.json"

with open(T_PRO_CREDS) as file:
    model_params = json.load(file)

llm = ChatOpenAI(**model_params)

# Utils

In [4]:
def create_model_prompts(system_prompt: str,
                         user_prompt: str) -> ChatPromptTemplate:
    system_prompt = SystemMessagePromptTemplate.from_template(system_prompt)
    user_prompt = HumanMessagePromptTemplate.from_template(user_prompt)
    chat_prompt = ChatPromptTemplate.from_messages(
        [system_prompt,
         user_prompt]
    )
    return chat_prompt

# Релизация

## Модель для извлечения ответа.

In [11]:
class CompareListResponse(BaseModel):
    lost_fragments: List[str] = Field(..., description="Фрагменты строки, которые были потеряны")
    added_fragments: List[str] = Field(..., description="Фрагменты, которые были добавлены в аугментированную строку")

parser = PydanticOutputParser(pydantic_object=CompareListResponse)

## Промпт

In [32]:
SYSTEM_PROMPT = """
Дана оригинальная строка str1 и аугментированная строка str2. Необходимо:  
1. Найти фрагменты из str1, которые потерялись при аугментации (отсутствуют в str2).  
2. Найти фрагменты из str2, которые добавились при аугментации (отсутствуют в str1).    

Примеры:
{examples}

Правила:
- Если фрагмент сокращен при аугментации, то он не потерян.

{format_instructions}
""".strip()

USER_PROMPT = """
Оригинальная строка: {original_string}
Аугментированная строка: {augmented_string}

Сравни две эти строки.
"""

prompt = create_model_prompts(SYSTEM_PROMPT, USER_PROMPT)

## Примеры

In [36]:
examples = [
    {
        "original_string": "Батарея ExeGate HR 12-9 12 Вольт 9 Ач (EP129860RUS)",
        "augmented_string": "БАТАРЕЯ ДЛЯ ИБП EXEGATE HR 12-9, 12В",
        "lost": ["9 Ач", "(EP129860RUS)"],
        "added": ["ДЛЯ ИБП"]
    },
    {
        "original_string": "Игристое вино розовое Villa Cialdini Brut Rose 2021, 0.75 л, 12%, Италия, Эмилия-Романия, Pr.I.V.I. s.r.l., брют",
        "augmented_string": "Вино игристое Вилла Чальдини Розе 2021, 0,75 л, 12%, Италия, Эмилия-Романия, Пр.И.В.И. с.р.л., розовое, брют",
        "lost": [],
        "added": []
    },
    {
        "original_string": "Поддон деревянный, грузоподъемность 1500 кг, 1 сорт, (1200 x 800 мм)",
        "augmented_string": "Поддон деревянный 1500 кг БУ 1200х800мм",
        "lost": ["грузоподъемность", "1 сорт"],
        "added": ["БУ"]
    },
    {
        "original_string": "Картридж лазерный Хеrох 106R01277 оригинальный (двойная упаковка)",
        "augmented_string": "Картридж лазерный HEROCH 106R01277 черный ориг. ДУ",
        "lost": ["Хеrох"],
        "added": ["HEROCH", "черный"]
    }
]

examples_str = "\n".join([f"""
{i+1})
Оригинальная строка: {example["original_string"]}
Аугментированная строка: {example["augmented_string"]}
Ответ: {json.dumps({
    "lost_fragments": example["lost"],
    "added_fragments": example["added"]
}, ensure_ascii=False)}
""".strip() for i, example in enumerate(examples)])
print(examples_str)

1)
Оригинальная строка: Батарея ExeGate HR 12-9 12 Вольт 9 Ач (EP129860RUS)
Аугментированная строка: БАТАРЕЯ ДЛЯ ИБП EXEGATE HR 12-9, 12В
Ответ: {"lost_fragments": ["9 Ач", "(EP129860RUS)"], "added_fragments": ["ДЛЯ ИБП"]}
2)
Оригинальная строка: Игристое вино розовое Villa Cialdini Brut Rose 2021, 0.75 л, 12%, Италия, Эмилия-Романия, Pr.I.V.I. s.r.l., брют
Аугментированная строка: Вино игристое Вилла Чальдини Розе 2021, 0,75 л, 12%, Италия, Эмилия-Романия, Пр.И.В.И. с.р.л., розовое, брют
Ответ: {"lost_fragments": [], "added_fragments": []}
3)
Оригинальная строка: Поддон деревянный, грузоподъемность 1500 кг, 1 сорт, (1200 x 800 мм)
Аугментированная строка: Поддон деревянный 1500 кг БУ 1200х800мм
Ответ: {"lost_fragments": ["грузоподъемность", "1 сорт"], "added_fragments": ["БУ"]}
4)
Оригинальная строка: Картридж лазерный Хеrох 106R01277 оригинальный (двойная упаковка)
Аугментированная строка: Картридж лазерный HEROCH 106R01277 черный ориг. ДУ
Ответ: {"lost_fragments": ["Хеrох"], "added_

# Использование

In [37]:
compare_chain = prompt | llm

In [39]:
original_item = "Чай травяной Imperial Tea Collection 20 пакетиков (имбирь, анис, корица)"
augmented_item = "Чай травяной IMPERIAL TEA COLLECTION 20П имбирь, анис, корица"

print(compare_chain.invoke({
    "examples": examples_str,
    "format_instructions": parser.get_format_instructions(),
    "original_string": original_item,
    "augmented_string": augmented_item
}).content)

Для сравнения строк, нужно определить утраченные и добавленные фрагменты с учетом правил:

1. **Оригинальная строка**: "Чай травяной Imperial Tea Collection 20 пакетиков (имбирь, анис, корица)"
2. **Аугментированная строка**: "Чай травяной IMPERIAL TEA COLLECTION 20П имбирь, анис, корица"

### Утраченные фрагменты:
- "пакетиков" (заменено на "20П", что считается сокращением, а не потерей)
- "(имбирь, анис, корица)" (добавлены без скобок, считается фрагментом, который изменился, но не потерян, так как содержание не было удалено)

Таким образом, фактически утраченных фрагментов нет, поскольку сокращение "20 пакетиков" → "20П" и изменение формата записи ингредиентов не считаются потерей содержания.

### Добавленные фрагменты:
- "IMPERIAL TEA COLLECTION" в верхнем регистре. Однако, если считать, что это просто изменение регистра, то данный фрагмент не считается добавленным.
- Нет новых добавленных фрагментов, поскольку все остальные элементы присутствуют в оригинальной строке.

Таким образ